In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
%load_ext autoreload
%autoreload 2
#useful for displaying wide data frames
pd.set_option('display.max_columns', 50)

In [2]:
#load the data into Pandas dataframes
df_market = pd.read_csv("input/markets.csv")
df_runners = pd.read_csv("input/runners.csv",dtype={'barrier': np.int16,'handicap_weight': np.float16})

#for my simple model, I'm ignoring other columns. I recommend starting with form if you're looking to add features
#df_odds = pd.read_csv("../input/odds.csv")
#df_form = pd.read_csv("../input/forms.csv")
#df_condition = pd.read_csv("../input/conditions.csv")
#df_weather = ("../input/weather.csv")
#df_rider = ("../input/riders.csv")
#df_horse = ("../input/horses.csv")
#df_horse_sex = ("../input/horse_sexes.csv")

In [120]:
##merge the runners and markets data frames
df_runners_and_market = pd.merge(df_runners,df_market,left_on='market_id',right_on='id',how='outer')
#df_runners_and_market.index = df_runners_and_market['id_x'] 
df_runners_and_market = df_runners_and_market.rename(columns={'id_x': 'runner_id'})

In [121]:
df_runners_and_market.head()

,runner_id,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,...,tip_four_win,tip_four_place,tip_five_win,tip_five_place,tip_six_win,tip_six_place,tip_seven_win,tip_seven_place,tip_eight_win,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place
0,4.0,2016-06-26 18:54:31.800293,1.0,NaN,1.0,NaN,4.0,4.0,4.0,58.5,4.0,10.0,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,...,t,t,t,t,f,t,f,t,f,t,t,t,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
1,10.0,2016-06-26 18:54:31.974395,1.0,NaN,0.0,NaN,10.0,4.0,10.0,56.5,10.0,11.0,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
2,5.0,2016-06-26 18:54:31.835329,1.0,NaN,0.0,NaN,5.0,5.0,5.0,56.5,5.0,5.0,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,...,f,f,f,t,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
3,6.0,2016-06-26 18:54:31.873492,1.0,NaN,0.0,NaN,6.0,6.0,6.0,56.5,6.0,12.0,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
4,1.0,2016-06-26 18:54:31.746854,1.0,NaN,0.0,NaN,1.0,1.0,1.0,58.5,1.0,8.0,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0


In [124]:
numeric_features = ['position', 'horse_id', 'runner_id', 'market_id','barrier','handicap_weight']
categorical_features = []

#convert to factors
'''for feature in categorical_features:
    df_runners_and_market[feature] = df_runners_and_market[feature].astype(str)
    df_runners_and_market[feature] = df_runners_and_market[feature].replace('nan','0') #have to do this because of a weird random forest bug

    df_features = df_runners_and_market[numeric_features]

for feature in categorical_features:
    encoded_features = pd.get_dummies(df_runners_and_market[feature])
    encoded_features.columns = feature + encoded_features.columns
    df_features = pd.merge(df_features,encoded_features,left_index=True,right_index=True,how='inner') 
'''
df_features = df_runners_and_market[numeric_features]
#turn the target variable into a binary feature: did or did not win
df_features['win'] = False
df_features.loc[df_features['position'] == 1,'win'] = True
#df_features = df_features.reset_index().rename(columns={'id_x': 'horse_id'})
df_features = df_features.reindex(np.random.permutation(df_features.index))
#del df_runners_and_market, encoded_features, df_features['position']

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [89]:
def getInd(race):
    won = race[race.win == True]
    if won.empty:
        wonClass = 8888
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) 
    return wonClass
.groupby('market_id').apply(getInd)

In [128]:
runnerMap = {}

def createRaceFinal(race):
    # we'll need the class of each horse later, which we can get from the indices of this list
    runnerMap[race.iloc[0,:].market_id] = race.runner_id.values
    
    
    won = race[race.win == True]
    if won.empty:
        won = 8888
        wonClass = 8888
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) # get what class the winning horse will be    race = race.drop('win', axis=1) # don't need it anymore
    
    race = race.drop(['horse_id', 'win','position', 'runner_id', 'market_id'], axis=1)
    st = race.stack().to_frame().T #stack it
    
    
    #rename the columns
    # could probably lose this
    st.columns = ['{}_{}'.format(col[1], col[0]) for col in st.columns]
    vals = st.values
    
    #add fake columns to create features for 25 horses
    colsToAdd = (race.shape[1] * 25) - st.shape[1]
    
    ones = (np.ones(colsToAdd) * -999)[:, np.newaxis].T

    withAdded = np.concatenate((vals, ones ), axis=1)
    withAdded = np.append(withAdded, won)
    withAdded = np.append(withAdded, wonClass)
    return withAdded

In [129]:
data = df_features.groupby('market_id').apply(createRaceFinal)

In [137]:
vals = np.vstack(data.values)
valsMask = vals[:,-1] < 8888
vals = vals[valsMask]

In [141]:
X = vals[:, 0:-2]
y = vals[:, -1]

In [110]:
allOdds = pd.read_csv("input/odds.csv")

In [145]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [146]:
probs = lr.predict_proba(X)

In [147]:
probs.shape

(6365, 21)

In [16]:
from runRaces import runRaces
from sklearn.linear_model import LogisticRegression

p = runRaces(df_f, LogisticRegression(), allOdds,n_trials=3)

trial 0
position           7651
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64


/Users/ryankresse/Documents/py/horses/runRaces.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_test['winProb'] = predictions


Total bets: 10200
Total wins: 250
trial 1
position           7571
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64
Total bets: 10293
Total wins: 297
trial 2
position           7313
market_id             0
barrier               0
handicap_weight       0
win                   0
dtype: int64
Total bets: 10048
Total wins: 259
mean payout: -27.929999999999996, std payout: 5.752257527846494


In [15]:
%reload_ext autoreload